# PASCAL VOC - Light-Weight Refinenet

## 20 semantic classes + background

### Light-Weight RefineNet based on ResNet-50/101/152 and MobileNet-v2

In [0]:
# Download and unpack Github's code / install python packages
!wget https://github.com/DrSleep/light-weight-refinenet/archive/master.zip
!unzip master.zip
!pip3 install -r light-weight-refinenet-master/requirements3.txt

--2019-04-10 12:38:01--  https://github.com/DrSleep/light-weight-refinenet/archive/master.zip
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/DrSleep/light-weight-refinenet/zip/master [following]
--2019-04-10 12:38:01--  https://codeload.github.com/DrSleep/light-weight-refinenet/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.253.121, 192.30.253.120
Connecting to codeload.github.com (codeload.github.com)|192.30.253.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip.2’

master.zip.2            [ <=>                ]   3.54M  21.3MB/s    in 0.2s    

2019-04-10 12:38:01 (21.3 MB/s) - ‘master.zip.2’ saved [3712931]

Archive:  master.zip
6364d27fbf16fca41a9a77840b49c3c672f6ca7e
replace light-weight-refinenet-m

KeyboardInterrupt: ignored

In [0]:
import six
import sys
sys.path.append('light-weight-refinenet-master/')

from models.mobilenet import mbv2
from models.resnet import rf_lw50, rf_lw101, rf_lw152
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from utils.helpers import prepare_img

In [0]:
%matplotlib inline

import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch

from PIL import Image

In [0]:
cmap = np.load('light-weight-refinenet-master/utils/cmap.npy')
has_cuda = torch.cuda.is_available()
anuepath="/content/drive/My Drive/anue_eval"
Real=glob.glob(anuepath+'/leftImg8bit/*_leftImg8bit.png')
Fake=glob.glob(anuepath+'/generated/*_generated.jpg')

fakereal=list(zip(sorted(Fake), sorted(Real)))
print(fakereal[0])
n_classes = 60

('/content/drive/My Drive/anue_eval/generated/000035_generated.jpg', '/content/drive/My Drive/anue_eval/leftImg8bit/000035_leftImg8bit.png')


In [0]:
# Initialise models
model_inits = { 
    'rf_lw101_context'   : rf_lw101, # key / constructor
    }

models = dict()
for key,fun in six.iteritems(model_inits):
    net = fun(n_classes, pretrained=True).eval()
    if has_cuda:
        net = net.cuda()
    models[key] = net

In [0]:
# Figure 2 from the paper
n_cols = len(models) + 1 # 1 - for image, 1 - for GT
from google.colab.patches import cv2_imshow
plt.figure(figsize=(16, 12))
idx = 0
totalacc=0
with torch.no_grad():
    for img_path in fakereal:
        imgf = np.array(cv2.resize(cv2.imread(img_path[0]), (1024, 768)))[:, :, ::-1]
        imgr = np.array(cv2.resize(cv2.imread(img_path[1])[:-3], (1024, 768)))[:, :, ::-1]
        
        orig_sizef = imgf.shape[:2][::-1]
        orig_sizer = imgr.shape[:2][::-1]
        
        img_inpf = torch.tensor(prepare_img(imgf).transpose(2, 0, 1)[None]).float()
        if has_cuda:
            img_inpf = img_inpf.cuda()
        img_inpr = torch.tensor(prepare_img(imgr).transpose(2, 0, 1)[None]).float()
        if has_cuda:
            img_inpr = img_inpr.cuda()
        
        idx+=1
        
        for mname, mnet in six.iteritems(models):
            segmf = mnet.cuda()(img_inpf)[0].data.cpu().numpy().transpose(1, 2, 0)
            segmf = cv2.resize(segmf, orig_sizef, interpolation=cv2.INTER_CUBIC)
            segmf = cmap[segmf.argmax(axis=2).astype(np.uint8)]
            
            segmr = mnet.cuda()(img_inpr)[0].data.cpu().numpy().transpose(1, 2, 0)
            segmr = cv2.resize(segmr, orig_sizer, interpolation=cv2.INTER_CUBIC)
            segmr = cmap[segmr.argmax(axis=2).astype(np.uint8)]
            
            
            #calculate differences
            segmf=np.array(segmf)
            segmr=np.array(segmr)
            diff=np.absolute(segmr-segmf)
            accperc=(np.count_nonzero(diff.flatten()))/(len(diff.flatten()))
            
              
             
            totalacc+=accperc
accuracy=totalacc/idx
print(accuracy)     

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


0.4049297155688293


<Figure size 1152x864 with 0 Axes>

In [0]:
totalacc/idx

0.4049297155688293

Light-Weight RefineNet for non-commercial purposes

Copyright (c) 2018, Vladimir Nekrasov

All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

    Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

    Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
